# Answers


# Homework 1 - Wikipedia Web Traffic Time Series


У вас есть данные по посещению 1000 страниц  Википедии из разных стран и разных девайсов ( \*  * данные взяты из [Kaggle соревнования](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train* и *wikipedia_test* - содержат данные о трафике. Это файлы csv, где каждая строка соответствует определенной статье, и каждый столбец соответствует конкретной дате. В некоторых записях отсутствуют данные. Названия страниц содержат проект Википедии (например, en.wikipedia.org), тип доступа (например, desktop) и тип агента (например, spider). Другими словами, каждое имя статьи имеет следующий формат: «name_project_access_agent» (например, «AKB48_zh.wikipedia.org_all-access_spider»).

Вам нужно ответить на [вопросы](https://docs.google.com/forms/d/e/1FAIpQLSfDjWeeZJw5EvmKn1x_6b9xicjn7ed3MF0rbNm4Cmwr7psSkQ/viewform?usp=sf_link) и попробовать сделать самую простую модель которая сможет предсказывать будущие посещения. 

Вот примеры временных рядов посещаемости страниц Википедии (*синие* - обучающая выборка, *зеленые* - предсказания модели победителя соревнования на Kaggle, *оранжевые* - реальные значения):
![Wikipedia Web Traffic Time Series](https://image.ibb.co/cUpEJa/predictions.png)

In [1]:
import re
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("../data/wikipedia_train.csv")
test = pd.read_csv("../data/wikipedia_test.csv")

## Data Analysis

In [3]:
def get_language(page):
    res = re.search('[a-z][a-z].wikipedia.org',page)
    if res:
        return res.group(0)[0:2]
    return 'na'

### 1 - Сколько страниц из русской Википедии в датасете?

In [7]:
train['Page'].map(get_language).value_counts()

en    140
fr    128
ja    124
na    124
de    122
ru    102
es    101
zh     98
Name: Page, dtype: int64

** ru - 102 **

### 2 - Какая самая популярная страница русской Википедии (в среднем)?

In [5]:
train[train['Page'].map(get_language) == 'ru'].mean(axis=1).sort_values(ascending=False)[:5]

180    2171.240654
725    1873.478972
666    1856.995327
685    1787.502336
689    1708.794393
dtype: float64

In [6]:
train.loc[180]['Page']

'Facebook_ru.wikipedia.org_desktop_all-agents'

**Facebook_ru.wikipedia.org_desktop_all-agents**

## Forecasting

Нужно преобразовать `train` данные в следующий формат:

In [11]:
train.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-08-22,2016-08-23,2016-08-24,2016-08-25,2016-08-26,2016-08-27,2016-08-28,2016-08-29,2016-08-30,2016-08-31
0,15._November_de.wikipedia.org_desktop_all-agents,32.0,26.0,22.0,22.0,29.0,49.0,20.0,27.0,19.0,...,29.0,23.0,31.0,25.0,27.0,23.0,17.0,26.0,23.0,37.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,2.0,3.0,5.0,3.0,5.0,3.0,7.0,8.0,7.0,...,5.0,5.0,6.0,5.0,4.0,11.0,2.0,0.0,7.0,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,1.0,3.0,2.0,2.0,1.0,10.0,2.0,1.0,4.0,...,4.0,3.0,2.0,3.0,2.0,4.0,2.0,0.0,5.0,4.0
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,3.0,3.0,3.0,8.0,12.0,12.0,8.0,12.0,23.0,...,10.0,14.0,26.0,5.0,29.0,23.0,17.0,16.0,12.0,14.0
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,24.0,40.0,23.0,49.0,88.0,25.0,31.0,76.0,51.0,...,134.0,162.0,208.0,179.0,108.0,99.0,49.0,80.0,113.0,173.0


Таким образом у вас каждая сточка содержит набор фич (`Page`, `date`) и целевую переменную (`Visits`). Преобразовать данные в такой формат поможет функция `pd.melt()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

In [12]:
train = pd.melt(train, id_vars='Page', var_name='date', value_name='Visits')
train['date'] = train['date'].astype('datetime64[ns]')

test = pd.melt(test, id_vars='Page', var_name='date', value_name='Visits')
test['date'] = test['date'].astype('datetime64[ns]')

Оценивать качество предсказаний мы будем с помощью [SMAPE](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error) :

In [16]:
def pandas_smape(df):
    df.fillna(0, inplace=True)
    df["SMAPE"] = 200 * np.abs(df["Visits"] - df["pred_Visits"]) / (df["Visits"] + df["pred_Visits"])
    df["SMAPE"].fillna(0, inplace=True)
    return np.mean(df["SMAPE"])

### Last day baseline

Нужно сделать прогноз на основе посещений в последний известный нам день из train (продублировать значение для каждого дня в test)

### 3 - Значение SMAPE для предсказаний на основе последнего дня

In [17]:
train['date'].max()

Timestamp('2016-08-31 00:00:00')

In [18]:
last_day_baseline = train[train["date"] == '2016-08-31'].copy()

In [23]:
last_day_baseline = train[train["date"] == '2016-08-31'].copy()
last_day_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)
last_day_baseline.drop("date", axis=1, inplace=True)

In [24]:
pandas_smape(test.merge(last_day_baseline, on="Page", how='left'))

54.16127748085961

**54.16**

### Median baseline

Нужно сделать прогноз на основе медианы за последние **30** дней из `train`. 

А затем улучшить предсказания используя информацию выходной это или нет (вам поможет функция [dayofweek](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DatetimeIndex.dayofweek.html) ) и разные окна для подсчета медианы (7 дней, 60 дней и тд)

Вам поможет функция `pd.groupby()` (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

### 4 - Значение SMAPE для предсказаний на основе медианы за последние 30 дней

In [27]:
median_baseline = train.loc[train["date"] > '2016-08-01'].groupby(['Page']).median().reset_index()
median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [28]:
median_baseline.head()

,Page,pred_Visits
0,15._November_de.wikipedia.org_desktop_all-agents,24.0
1,2012_(film)_fr.wikipedia.org_all-access_spider,5.0
2,2016_FIFA_U-20女子ワールドカップ_ja.wikipedia.org_all-a...,2.5
3,2016_UEFA_Europa_League_Final_en.wikipedia.org...,15.5
4,2016_in_video_gaming_en.wikipedia.org_all-acce...,110.5


In [29]:
pandas_smape(test.merge(median_baseline, on="Page", how='left'))

52.465883293370794

In [30]:
train['Visits'].fillna(0, inplace=True)

In [31]:
median_baseline = train.loc[train["date"] > '2016-08-01'].groupby(['Page']).median().reset_index()
median_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [32]:
pandas_smape(test.merge(median_baseline, on="Page", how='left'))

51.89889768202216

**51.89**

### 5 - Попробуйте разные окна для подсчета медианы (7 дней, 60 дней и тд) и информацию про выходные

In [33]:
train["weekend"] = ((train["date"].dt.dayofweek) // 5 == 1).astype(int)
test["weekend"] = ((test["date"].dt.dayofweek) // 5 == 1).astype(int)

In [34]:
median_weekend_baseline = train.loc[train["date"] > '2016-08-01'].groupby(['Page', 'weekend']).median().reset_index()
median_weekend_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)

In [35]:
pandas_smape(test.merge(median_weekend_baseline, on=["Page", "weekend"], how='left'))

51.61290061039105

In [36]:
from dateutil.relativedelta import relativedelta

In [37]:
windows = [7, 10, 12, 14, 25, 30, 60, 120, 240, 360, 365]

In [38]:
for w in windows:
    start_day = train['date'].max() - relativedelta(days=w)
    median_weekend_baseline = train.loc[train["date"] > start_day].groupby(['Page', 'weekend']).median().reset_index()
    median_weekend_baseline.rename(columns={"Visits": "pred_Visits"}, inplace=True)
    print(w, pandas_smape(test.merge(median_weekend_baseline, on=["Page", "weekend"], how='left')))

7 50.77751305801209
10 50.490014167944764
12 50.263926861645594
14 50.36151445825095
25 51.72298341605937
30 51.61290061039105
60 51.275054111529066
120 49.67972366075196
240 48.83413376317465
360 49.02014812609877
365 49.02461134161452


Best - **240** days **48.83**